In [10]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Missing').getOrCreate()

In [11]:
#cargar datos
training = spark.read.csv('test1.csv',header=True,inferSchema=True)
#ver nombres columnas
training.columns

['Name', 'age', 'Experience', 'Salary']

In [12]:
#age y experience columna utilizadas para la prediccion, se ponen en vector
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["age","Experience"],outputCol="Independent Features")
output=featureassembler.transform(training)
output.show()
output.columns

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [13]:
#seleccionar columnas
finalized_data=output.select("Independent Features","Salary")

In [14]:
#tenemos los atributos en un vector y la clase a predecir
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [15]:
from pyspark.ml.regression import LinearRegression
#separamos datos
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
#decimos cuales son los atributos y cual la clase
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor=regressor.fit(train_data)

In [16]:
#coeficientes
regressor.coefficients
#interceptor
regressor.intercept

16470.03994673731

In [17]:
#predicion
pred_results=regressor.evaluate(test_data)

In [18]:
#ver resultados
pred_results.predictions.show()

+--------------------+------+-----------------+
|Independent Features|Salary|       prediction|
+--------------------+------+-----------------+
|          [30.0,8.0]| 25000|26903.59520639148|
+--------------------+------+-----------------+



In [19]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(1903.595206391481, 3623674.709796625)